In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

## Load data and basic EDA

In [2]:
# Source: https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data
patients = pd.read_csv("heart_failure_clinical_records_dataset.csv")

In [3]:
patients.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [5]:
patients["DEATH_EVENT"].value_counts()

DEATH_EVENT
0    203
1     96
Name: count, dtype: int64

In [6]:
patients.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Benchmark: Logistic Regression

- Stratified train/test split
- Scaled continuous features

In [7]:
X = patients.drop(columns="DEATH_EVENT")
y = patients["DEATH_EVENT"]

In [8]:
X.shape, y.shape

((299, 12), (299,))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((224, 12), (75, 12), (224,), (75,))

In [11]:
y_train.mean(), y_test.mean()

(0.32142857142857145, 0.32)

In [12]:
scalable_features = [
    "age",
    "creatinine_phosphokinase",
    "ejection_fraction",
    "platelets",
    "serum_creatinine",
    "serum_sodium",
    "time"
]

In [13]:
scaler = preprocessing.StandardScaler().fit(X_train[scalable_features])

In [14]:
X_train[scalable_features] = scaler.transform(X_train[scalable_features])
X_test[scalable_features] = scaler.transform(X_test[scalable_features])

In [15]:
X_train[scalable_features].describe()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time
count,2.240000e+02,2.240000e+02,2.240000e+02,2.240000e+02,2.240000e+02,2.240000e+02,2.240000e+02
mean,2.379049e-16,-3.568574e-17,-2.379049e-17,1.903239e-16,-8.326673e-17,1.891344e-15,1.427430e-16
std,1.002240e+00,1.002240e+00,1.002240e+00,1.002240e+00,1.002240e+00,1.002240e+00,1.002240e+00
min,-1.793846e+00,-5.508546e-01,-2.079043e+00,-2.402115e+00,-9.120501e-01,-5.468987e+00,-1.657827e+00
25%,-8.456116e-01,-4.632020e-01,-6.597497e-01,-5.670978e-01,-5.034108e-01,-6.214055e-01,-7.615072e-01
50%,-6.978372e-02,-3.326971e-01,4.989703e-02,-8.287122e-02,-2.990911e-01,7.110610e-02,-2.078978e-01
75%,7.922473e-01,-6.434811e-03,6.708379e-01,4.215315e-01,3.292830e-02,7.636177e-01,9.652270e-01
max,2.516309e+00,7.082709e+00,2.178837e+00,5.919521e+00,8.180174e+00,2.610315e+00,2.019721e+00


In [16]:
X_train[[c for c in X_train.columns if c not in scalable_features]].describe()

,anaemia,diabetes,high_blood_pressure,sex,smoking
count,224.000000,224.000000,224.000000,224.000000,224.000000
mean,0.433036,0.419643,0.370536,0.669643,0.330357
std,0.496605,0.494606,0.484030,0.471395,0.471395
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
LR_clf = LogisticRegression()
LR_clf.fit(X_train, y_train)

LogisticRegression()

In [18]:
LR_clf.score(X_test, y_test)

0.88

In [19]:
y_pred = LR_clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[49,  2],
       [ 7, 17]])

In [20]:
tn, fp, fn, tp = cm.ravel()
print(f"True positives: {tp}")
print(f"True negatives: {tn}")
print(f"False positives: {fp}")
print(f"False negatives: {fn}")

True positives: 17
True negatives: 49
False positives: 2
False negatives: 7
